In [14]:
import pandas as pd
import os
import tqdm
from collections import Counter
import numpy as np
from scipy.stats import multinomial
import numpy as np
import ast


In [ ]:
# Load the data
df = pd.read_csv("/Users/reem/Mov/nextclade_results/GtoA_LLRs_final.tsv",sep="\t")
#df = df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)
df.to_csv("/Users/reem/Mov/nextclade_results/GtoA_LLRs_final.tsv",sep="\t", index=False)



In [44]:
df.head()

,seqName,privateNucMutations.unlabeledSubstitutions,subs,Counts,context,spectrum,G>A_context_counts,proportions,G>A_llr
0,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,"G204A,C2445T,C4331T,C5621T,C5622T,C6633T,C1240...","G>A,C>T,C>T,C>T,C>T,C>T,C>T,G>A,C>T,A>C,A>T,G>...","{'G>A': 3, 'C>T': 8, 'A>C': 1, 'A>T': 1, 'G>T'...","CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...","C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]...","{'C[G>A]T': 1, 'A[G>A]G': 1, 'T[G>A]C': 1}","{'C[G>A]T': 0.3333333333333333, 'A[G>A]G': 0.3...",3.056241
1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,NaN,NaN,{},NaN,NaN,{},{},0.000000
2,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,NaN,NaN,{},NaN,NaN,{},{},0.000000
3,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,G19999T,G>T,{'G>T': 1},AGT,A[G>T]T,{},{},0.000000
4,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,"G410T,A7881G","G>T,A>G","{'G>T': 1, 'A>G': 1}","TGG,AAT","T[G>T]G,A[A>G]T",{},{},0.000000


In [ ]:
# Convert the string representation of dicts back to actual dicts
import ast
df["Counts"] = df["Counts"].apply(ast.literal_eval)
print(type(df["Counts"]))

In [ ]:
# Calculate LLR
probs_df=pd.read_csv("/Users/reem/Downloads/estimated_mutation_distribution.tsv", delimiter="\t")
pM=probs_df["Molnupiravir"].to_numpy(dtype=float)
pN=probs_df["Normal"].to_numpy(dtype=float)
mut_types=probs_df["MutationType"].str.replace("→",">").tolist()

def get_likelihood_ratio(counts,pM,pN):
    counts=np.array(counts,dtype=float)
    llM= float(multinomial.logpmf(counts, n=np.sum(counts), p=pM))
    llN = float(multinomial.logpmf(counts, n=np.sum(counts), p=pN))
    llr=llM-llN
    return llr
llr_list = []
for counts_dict in df["Counts"]:
    counts = [counts_dict.get(mt, 0) for mt in mut_types]
    llr = get_likelihood_ratio(counts,pM,pN)
    llr_list.append(llr)
df["LLR"] = llr_list



In [ ]:
#Get Counts per substitution context
def count_AtoT(spectrum):
    counts = Counter()
    if not isinstance(spectrum, str):
        return {}
    muts = spectrum.split(",")
    for mut in muts:
        if mut[2:5] == 'A>T':
            counts[mut]+=1
    return counts

def count_CtoA(spectrum):
    counts = Counter()
    if not isinstance(spectrum, str):
        return {}
    muts = spectrum.split(",")
    for mut in muts:
        if mut[2:5] == 'C>A':
            counts[mut]+=1
    return counts

def count_GtoT(spectrum):
    counts = Counter()
    if not isinstance(spectrum, str):
        return {}
    muts = spectrum.split(",")
    for mut in muts:
        if mut[2:5] == 'G>T':
            counts[mut]+=1
    return counts

def count_TtoG(spectrum):
    counts = Counter()
    if not isinstance(spectrum, str):
        return {}
    muts = spectrum.split(",")
    for mut in muts:
        if mut[2:5] == 'T>G':
            counts[mut]+=1
    return counts

df["A>T_counts"] = df["spectrum"].apply(count_AtoT)
df["C>A_counts"] = df["spectrum"].apply(count_CtoA)
df["G>T_counts"] = df["spectrum"].apply(count_GtoT)
df["T>G_counts"] = df["spectrum"].apply(count_TtoG)
print(df["A>T_counts"].head())
print(df["C>A_counts"].head())
print(df["G>T_counts"].head())
print(df["T>G_counts"].head())


0    {}
1    {}
2    {}
3    {}
4    {}
Name: T>G_counts, dtype: object


In [ ]:
# Get Proportions per substitution context
def get_proportion(df):
    dict = {}
    total = sum(df.values())
    for key, value in df.items():
        dict[key] = value/total
       
    return dict

df["A>Tproportions"] = df.apply(lambda row: get_proportion(row["A>T_counts"]), axis=1)
df["C>Aproportions"] = df.apply(lambda row: get_proportion(row["C>A_counts"]), axis=1)
df["G>Tproportions"] = df.apply(lambda row: get_proportion(row["G>T_counts"]), axis=1)
df["T>Gproportions"] = df.apply(lambda row: get_proportion(row["T>G_counts"]), axis=1)

print(df["A>Tproportions"].head(10))
print(df["C>Aproportions"].head(10))
print(df["G>Tproportions"].head(10))
print(df["T>Gproportions"].head(10))


0                  {}
1                  {}
2                  {}
3                  {}
4                  {}
5                  {}
6                  {}
7                  {}
8                  {}
9    {'T[T>G]A': 1.0}
Name: T>Gproportions, dtype: object


In [ ]:
def generate_all_possible_contexts(sub):
    bases = ["A","C","G","T"]
    possible_contexts = []
    for a in bases:
        for b in bases:
            possible_contexts.append(f"{a}[{sub}]{b}")
    return possible_contexts
print(generate_all_possible_contexts("A>G"))

['A[T>C]A', 'A[T>C]C', 'A[T>C]G', 'A[T>C]T', 'C[T>C]A', 'C[T>C]C', 'C[T>C]G', 'C[T>C]T', 'G[T>C]A', 'G[T>C]C', 'G[T>C]G', 'G[T>C]T', 'T[T>C]A', 'T[T>C]C', 'T[T>C]G', 'T[T>C]T']


In [ ]:
# Compute mean context probabilities
def get_mean_context_probs(df, llr_condition, prop_col, llr_col='LLR'):
    """
    Filters dataframe into llr>6 and llr<6 and computes mean 
    proportions per substitution per context

    Returns a flattened dataframe with mutational contexts,seqname,LLR
    and computed means as the last row

    """

    df_filtered = df[llr_condition(df[llr_col])].copy()
    
    pivot = pd.json_normalize(df_filtered[prop_col]).fillna(0)
    pivot = pivot.loc[(pivot != 0).any(axis=1)]

    return pivot.mean()

#print(get_mean_context_probs(df, llr_condition=lambda x: x > 1, prop_col="T>Cproportions", llr_col='LLR'))
def build_prob_table(df,prop_col,llr_high,llr_low,llr_col='LLR'):
    """
    Builds a table of mean context probabilities (Molnupiravir vs Normal)
    for a specific substitution type.
    """
    llr_high=6
    llr_low=6

    Mov_probs = get_mean_context_probs(df.loc[(df!=0).any(axis=1)], lambda x: x > 6, prop_col='T>Gproportions')
    Normal_probs  = get_mean_context_probs(df.loc[(df!=0).any(axis=1)], lambda x: x < 6, prop_col='T>Gproportions')
    all_contexts = generate_all_possible_contexts("T>G")
    
    # Match means to all possible contexts
    Mov_probs = Mov_probs.reindex(all_contexts, fill_value=0)
    Normal_probs = Normal_probs.reindex(all_contexts, fill_value=0)

    print(Mov_probs)
    print(Normal_probs)

    df_prob = pd.DataFrame({'Mutational_Context':all_contexts,
                           'Molnupiravir': Mov_probs.values,
                            'Normal': Normal_probs.values
                            }) 

    
    return df_prob


df_prob = build_prob_table(df,prop_col='T>Gproportions',llr_high=6, llr_low=6, llr_col='LLR')

print(df_prob)



A[T>G]A    0.000000
A[T>G]C    0.000000
A[T>G]G    0.000000
A[T>G]T    0.000000
C[T>G]A    0.000000
C[T>G]C    0.333333
C[T>G]G    0.000000
C[T>G]T    0.000000
G[T>G]A    0.000000
G[T>G]C    0.000000
G[T>G]G    0.000000
G[T>G]T    0.333333
T[T>G]A    0.000000
T[T>G]C    0.333333
T[T>G]G    0.000000
T[T>G]T    0.000000
dtype: float64
A[T>G]A    0.047627
A[T>G]C    0.060305
A[T>G]G    0.037864
A[T>G]T    0.080590
C[T>G]A    0.017799
C[T>G]C    0.080583
C[T>G]G    0.066506
C[T>G]T    0.175690
G[T>G]A    0.028940
G[T>G]C    0.027340
G[T>G]G    0.027145
G[T>G]T    0.061652
T[T>G]A    0.024779
T[T>G]C    0.092241
T[T>G]G    0.046293
T[T>G]T    0.124646
dtype: float64
   Mutational_Context  Molnupiravir    Normal
0             A[T>G]A      0.000000  0.047627
1             A[T>G]C      0.000000  0.060305
2             A[T>G]G      0.000000  0.037864
3             A[T>G]T      0.000000  0.080590
4             C[T>G]A      0.000000  0.017799
5             C[T>G]C      0.333333  0.080583
6       

In [98]:
print(df_prob)
print(df_prob["Molnupiravir"].sum())
print(df_prob["Normal"].sum())

   Mutational_Context  Molnupiravir    Normal
0             A[T>G]A      0.000000  0.047627
1             A[T>G]C      0.000000  0.060305
2             A[T>G]G      0.000000  0.037864
3             A[T>G]T      0.000000  0.080590
4             C[T>G]A      0.000000  0.017799
5             C[T>G]C      0.333333  0.080583
6             C[T>G]G      0.000000  0.066506
7             C[T>G]T      0.000000  0.175690
8             G[T>G]A      0.000000  0.028940
9             G[T>G]C      0.000000  0.027340
10            G[T>G]G      0.000000  0.027145
11            G[T>G]T      0.333333  0.061652
12            T[T>G]A      0.000000  0.024779
13            T[T>G]C      0.333333  0.092241
14            T[T>G]G      0.000000  0.046293
15            T[T>G]T      0.000000  0.124646
1.0
1.0


In [99]:
#Calculate LLRs using prob_table:

def calculate_llr(count_dict, pM, pN, contexts):
    counts = np.array([count_dict.get(ctx, 0) for ctx in contexts])
    n = counts.sum()
    if n == 0:
        return np.nan
    llM = multinomial.logpmf(counts, n=n, p=pM)
    llN = multinomial.logpmf(counts, n=n, p=pN)
    return float(llM - llN)

contexts = df_prob["Mutational_Context"].values.tolist()
pM = df_prob["Molnupiravir"].values.tolist()
pN = df_prob["Normal"].values.tolist()
df[f"T>G_llr"] = df["T>G_counts"].apply(lambda x: calculate_llr(x, pM, pN, contexts))

print(df[f"T>G_llr"])






0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
1970295   NaN
1970296   NaN
1970297   NaN
1970298   NaN
1970299   NaN
Name: T>G_llr, Length: 1970300, dtype: float64


In [100]:

df.head()

,seqName,privateNucMutations.unlabeledSubstitutions,subs,Counts,context,spectrum,G>A_context_counts,proportions,G>A_llr,LLR,...,C>Aproportions,G>Tproportions,T>Aproportions,A>T_llr,C>A_llr,G>T_llr,T>A_llr,T>G_counts,T>Gproportions,T>G_llr
0,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,"G204A,C2445T,C4331T,C5621T,C5622T,C6633T,C1240...","G>A,C>T,C>T,C>T,C>T,C>T,C>T,G>A,C>T,A>C,A>T,G>...","{'G>A': 3, 'C>T': 8, 'A>C': 1, 'A>T': 1, 'G>T'...","CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...","C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]...","{'C[G>A]T': 1, 'A[G>A]G': 1, 'T[G>A]C': 1}","{'C[G>A]T': 0.3333333333333333, 'A[G>A]G': 0.3...",3.056241,-0.378720,...,{},{'T[G>T]A': 1.0},{},0.252897,NaN,-0.626445,NaN,{},{},NaN
1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,NaN,NaN,{},NaN,NaN,{},{},0.000000,0.000000,...,{},{},{},NaN,NaN,NaN,NaN,{},{},NaN
2,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,NaN,NaN,{},NaN,NaN,{},{},0.000000,0.000000,...,{},{},{},NaN,NaN,NaN,NaN,{},{},NaN
3,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,G19999T,G>T,{'G>T': 1},AGT,A[G>T]T,{},{},0.000000,-1.509730,...,{},{'A[G>T]T': 1.0},{},NaN,NaN,0.112318,NaN,{},{},NaN
4,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,"G410T,A7881G","G>T,A>G","{'G>T': 1, 'A>G': 1}","TGG,AAT","T[G>T]G,A[A>G]T",{},{},0.000000,-1.720178,...,{},{'T[G>T]G': 1.0},{},NaN,NaN,-1.461764,NaN,{},{},NaN


In [101]:
df.to_csv("llrs_per_context.tsv", sep="\t", index=False) 

In [108]:
new_df = pd.read_csv("/Users/reem/Mov/nextclade_results/molnupiravir_misc/step_2_nextclade/llrs_only.tsv",sep="\t")

In [ ]:
#new_df=pd.concat([new_df, df[['C>T_llr','A>G_llr','A>T_llr','C>A_llr', 'G>T_llr', 'T>A_llr','T>G_llr']]],axis=1)
new_df=pd.concat([new_df, df[['T>C_llr']]],axis=1)
new_df.head()



,seqName,G>A_llr,C>T_llr,A>G_llr,A>T_llr,C>A_llr,G>T_llr,T>A_llr,T>G_llr,T>C_llr
0,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,3.056241,-1.193343,1.636224,0.252897,NaN,-0.626445,NaN,NaN,NaN
1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,0.000000,NaN,NaN,NaN,NaN,0.112318,NaN,NaN,NaN
4,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,0.000000,NaN,0.044808,NaN,NaN,-1.461764,NaN,NaN,NaN


In [ ]:
# Create final DataFrame using LLRs from estimated distributions and the newly calculated LLRs.
new_col = df['LLR']
new_df.insert(loc=1, column='LLR', value=new_col)
new_df.head()


,seqName,LLR,G>A_llr,C>T_llr,A>G_llr,A>T_llr,C>A_llr,G>T_llr,T>A_llr,T>G_llr,T>C_llr
0,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,-0.378720,3.056241,-1.193343,1.636224,0.252897,NaN,-0.626445,NaN,NaN,NaN
1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,-1.509730,0.000000,NaN,NaN,NaN,NaN,0.112318,NaN,NaN,NaN
4,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,-1.720178,0.000000,NaN,0.044808,NaN,NaN,-1.461764,NaN,NaN,NaN


In [ ]:
# SAVE FINAL DF

new_df.to_csv("llrs_only.tsv", sep="\t", index=False)